In [4]:
import dataclasses

import jax
import jax.numpy as jnp


### Jax Pallas Experimental Flash Attention

Github: https://github.com/jax-ml/jax/blob/922935a916dbcf599226f1ce3081feb6481328c3/jax/experimental/pallas/ops/gpu/attention.py

In [25]:
rng = jax.random.PRNGKey(0)

max_seq_len = 64
B, T, nH, C = 32, 15, 8, 128

q = jax.random.normal(rng, (B, T, nH, C), dtype=jnp.bfloat16)
k = jax.random.normal(rng, (B, T, nH, C), dtype=jnp.bfloat16)
v = jax.random.normal(rng, (B, T, nH, C), dtype=jnp.bfloat16)

In [15]:
from jax.experimental.pallas.ops.gpu.attention import mha

@dataclasses.dataclass(frozen=True, slots=True)
class BlockSizes:
  block_q: int
  block_k: int

pad_width = ((0, 0),  # no padding on the first dimension
             (0, max_seq_len-15),
             (0, 0),
             (0, 0))  # pad two zeros on the right side of the second dimension

q = jnp.pad(q, pad_width, mode='constant', constant_values=0)
k = jnp.pad(k, pad_width, mode='constant', constant_values=0)
v = jnp.pad(v, pad_width, mode='constant', constant_values=0)

block_sizes = BlockSizes(block_q=T, block_k=T)

y = mha(q, k, v, block_sizes=block_sizes, segment_ids=None, causal=True)

TypeError: got an unexpected keyword argument 'block_sizes'

### FlashAttention Jax

Github: https://github.com/nshepperd/flash_attn_jax

In [1]:
!pip install flash-attn-jax

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 MB 29.2 MB/s eta 0:00:00


In [30]:
from flash_attn_jax import flash_mha

# flash_mha : [n, l, h, d] x [n, lk, hk, d] x [n, lk, hk, d] -> [n, l, h, d]
%timeit flash_mha(q,k,v,softmax_scale=None, is_causal=True, window_size=(-1,-1))

393 µs ± 2.29 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Flash Attention - Jax
Github: https://github.com/lucidrains/flash-attention-jax

In [18]:
!pip install flash-attention-jax

In [31]:
from flash_attention_jax import causal_flash_attention

rng_key = jax.random.PRNGKey(42)

%timeit causal_flash_attention(q, k, v)

262 µs ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Kvax

Github: https://github.com/nebius/kvax

In [32]:
!pip install kvax

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 MB 21.7 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.33
    Uninstalling jaxlib-0.4.33:
      Successfully uninstalled jaxlib-0.4.33
  Attempting uninstall: jax
    Found existing installation: jax 0.4.33
    Uninstalling jax-0.4.33:
      Successfully uninstalled jax-0.4.33
